# Create OWL (PTP) Topology (MASTER)

## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

try:
    fm = fablib_manager()
                     
    fm.show_config()
except Exception as e:
    print(f"Exception: {e}")

### List slices

In [ ]:
fm.list_slices()

## Choose sites

### Choose ALL PTP-enalbed sites

In [ ]:
sites = fm.get_site_names()
print(f"all sites: {len(sites)} ({sites})")

ptp_sites = fm.list_sites(
    output="list",
    quiet=True,
    filter_function = lambda x:x['ptp_capable'] is True and x['state'] == 'Active',
)
ptp_site_names = [x['name'] for x in ptp_sites]

print(f"PTP-enabled and active sites: {len(ptp_site_names)} ({ptp_site_names})")


### If a certain number of sites (rather than ALL PTP-enabled sites) are desired

In [ ]:
# ptp_site_names = fm.get_random_sites(count=3, filter_function=lambda x: x['ptp_capable'] is True and x['state'] == 'Active') 

## Create a slice with one node placed on each PTP-enabled site

The following creates a slice with one node on each PTP-capable site, with basic NICs connected to FABRIC's FABnetv4 internet. 

In [ ]:
slice_name = "latency_monitoring_slice"

In [ ]:
slice = fm.new_slice(name=slice_name)

for i, site_name in enumerate(ptp_site_names):
    node = slice.add_node(name=f"node{i}", site=site_name, image='docker_rocky_8')
    node.add_fabnet()
    print(f"adding node{i} at {site_name}")

#Submit Slice Request
slice.submit();

## (Optional) Observe the Slice's Attributes

In [ ]:
try:
    slice = fm.get_slice(name=slice_name)
    slice.show()
    slice.list_nodes()
    slice.list_networks()
    slice.list_interfaces()
except Exception as e:
    print(f"Exception: {e}")

## Check connectivity via Experimenter's network

The following checks Node0 --> each node in the topology

In [ ]:
node0 = slice.get_node(name="node0")
nodes = slice.get_nodes()

for i in range(len(nodes)):
    dst_node_addr = nodes[i].get_interface(network_name=f'FABNET_IPv4_{nodes[i].get_site()}').get_ip_addr()
    stdout, stderr = node0.execute(f'ping -c 3 {dst_node_addr}')

    
# except Exception as e:
#     print(f"Exception: {e}")

# Prepare each node for time precision experiments

## Optimize repositories based on management network type (ipv4 vs ipv6)


In [ ]:
for node in nodes:
    IPv6Management = False
    ip_proto_index = "4"
    commands = "sudo ip -6 route del default via `ip -6 route show default|grep fe80|awk '{print $3}'` > /dev/null 2>&1"
    if node.validIPAddress(node.get_management_ip()) == "IPv6":
        IPv6Management = True
        ip_proto_index = "6"
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        commands = (
            f'sudo echo "max_parallel_downloads=10" |sudo tee -a /etc/dnf/dnf.conf;'
            f'sudo echo "fastestmirror=True" |sudo tee -a /etc/dnf/dnf.conf;'
            f'sudo echo "ip_resolve='
            + ip_proto_index
            + '" |sudo tee -a /etc/dnf/dnf.conf;'
        )
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        commands = (
            'sudo echo "Acquire::ForceIPv'
            + ip_proto_index
            + ' "true";" | sudo tee -a /etc/apt/apt.conf.d/1000-force-ipv'
            + ip_proto_index
            + "-transport"
        )
    if commands:
        stdout, stderr = node.execute(commands, quiet=True)
        print (f"Optimizing Repos for {node.get_name()}")
        #print (f"STDOUT: {stdout}")
        if stderr:
            print (f"STDERR: {stderr}")

## Set up PTP (Precision Time Protocol)

This block can take a while to execute. If successful, it will print `Installation of PTP Completed`.

In [ ]:
pre_requisites = None

clone_instructions = f"""
cd /tmp/;git clone --filter=blob:none --no-checkout --depth 1 --sparse https://github.com/fabric-testbed/MeasurementFramework.git;
cd /tmp/MeasurementFramework;git sparse-checkout add instrumentize/ptp/ansible;git checkout;
"""

ansible_instructions = f"""
cd /tmp/MeasurementFramework/instrumentize/ptp/ansible;
ansible-playbook --connection=local --inventory 127.0.0.1, --limit 127.0.0.1 playbook_fabric_experiment_ptp.yml;
"""

#Create execute threads
execute_threads = {}

for node in nodes:
    if [ele for ele in ["rocky", "centos"] if (ele in node.get_image())]:
        pre_requisites = f"""
        sudo dnf -y install epel-release ; sudo dnf -y install ansible git; mkdir /tmp/ptp_ansible/;
        """
    elif [ele for ele in ["ubuntu", "debian"] if (ele in node.get_image())]:
        pre_requisites = f"""sudo apt-get update;sudo apt-get -y install ansible git;"""
    else:
        pre_requisites = None
    print (f"Installing PTP on {node.get_name()}")
    execute_threads[node] = node.execute_thread(\
                f"{pre_requisites}"\
                f"{clone_instructions}"\
                f"{ansible_instructions}",\
                output_file=f"/tmp/{node.get_name()}_ptpinstall.log"\
                )

    #Wait for results from threads
for node,thread in execute_threads.items():
    print(f"Waiting for result from node {node.get_name()}")
    stdout,stderr = thread.result()
    #print(f"stdout: {stdout}")
    #print(f"stderr: {stderr}")
    #node.execute(f"{pre_requisites}"\
    #             f"{ansible_instructions}"\
    #            )

print (f"Installation of PTP Completed\n\n")

## Start Docker and verify it is running

`docker ps` should print `CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES`

In [ ]:
for node in nodes:
    node.execute("sudo systemctl start docker")
    node.execute("sudo systemctl enable docker")
    node.execute("sudo usermod -aG docker rocky")
    
    print(f"\n Verify installtion... on {node.get_name()}")
    node.execute("docker ps")

# (Optional) Extend the slice (Add 14 days)

In [ ]:
try:
    slice = fm.get_slice(name=slice_name)
    print(f"Lease End         : {slice.get_lease_end()}")
       
except Exception as e:
    print(f"Exception: {e}")

In [ ]:
import datetime

#Extend slice
end_date = (datetime.datetime.now().astimezone() + datetime.timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fm.get_slice(name=slice_name)
    slice.renew(end_date)
    print(f"New lease end date : {slice.get_lease_end()}")
    
except Exception as e:
    print(f"Exception: {e}")

# Delete the Slice

In [ ]:
# try:
#     slice = fm.get_slice(name=slice_name)
#     slice.delete()
# except Exception as e:
#     print(f"Exception: {e}")